# Street Standerdization

In [2]:
import numpy as np
import pandas as pd
from tabulate import tabulate
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from fuzzywuzzy import fuzz
import re
from pyproj import Proj, transform

In [3]:
filtered_df = pd.read_csv("../Dataset/New_Data/12_Loc_data_not_Available.csv", low_memory=False)

In [2]:
# def convert_coordinates(lon, lat):
#     in_proj = Proj(init='epsg:4326')  # Geographic coordinate system (WGS84)
#     out_proj = Proj(init='epsg:5235')  # SLD99 projected coordinate system

#     # Convert coordinates
#     x, y = transform(in_proj, out_proj, lon, lat)
#     return x, y


# x, y = convert_coordinates(79.84368, 6.937814)
# a, b = convert_coordinates(79.861053, 6.956226)
# c, d = convert_coordinates(79.864303, 6.943332)
# e, f = convert_coordinates(79.865935, 6.951743)

# print(x)
# print(y)
# print('---')
# print(a)
# print(b)
# print('---')
# print(c)
# print(d)
# print('---')
# print(e)
# print(f)
# print('---')

397224.67956068023
493135.9916141926
---
399148.56855434086
495168.5092086139
---
399504.9755588728
493741.8161293641
---
399687.119457731
494671.6750684878
---


c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\User\AppData\Local\Temp\ipykernel_68268\2847318522.py:6: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas

In [4]:
df2 = filtered_df  

# Function to preprocess and find the most similar street 
def preprocess_and_find_most_similar_street(street, street_list):
    max_similarity = 0
    most_similar_street = None
    for known_street in street_list:
        # Remove variations of "Road" using regular expressions 
        street_str = re.sub(r'(?i)(rd|road)', '', str(street)).strip().lower()
        known_street_str = re.sub(r'(?i)(rd|road)', '', str(known_street)).strip().lower()
        similarity = fuzz.ratio(street_str, known_street_str)
        if similarity > max_similarity:
            max_similarity = similarity
            most_similar_street = known_street
    return most_similar_street, max_similarity

clusters = {}

for index, row in df2.iterrows():
    street = row['street']
    
    # Find the most similar street in existing clusters
    found_cluster = False
    for cluster_key, cluster_value in clusters.items():
        most_similar_street, similarity = preprocess_and_find_most_similar_street(street, cluster_value)
        if similarity >= 90:  
            clusters[cluster_key].append(street)
            found_cluster = True
            break
    
    if not found_cluster:
        clusters[street] = [street]

street_mapping = {street: cluster[0] for cluster in clusters.values() for street in cluster}

df2.loc[:, 'standardized_street'] = df2['street'].map(street_mapping)

# Extract unique streets and their standardized versions
unique_streets = pd.DataFrame(list(set(street_mapping.values())), columns=['original_street'])
unique_streets['standardized_street'] = unique_streets['original_street'].map(street_mapping)

output_csv_path = '../Dataset/New_Data/unique_streets.csv'
unique_streets.to_csv(output_csv_path, index=False)

In [5]:
df2

,ad_id,UID,street,city,heading,desc,posted_date,price,price_type,price_land_pp,...,city_no,city_name,alt_name,city_lat,city_lon,g_city_lat,g_city_lon,main_city,main_city_id,standardized_street
0,245752,71194,Karshue Avenue,Colombo 8,6 perch land at Borella for sale,The perfect location to build your dream home ...,5/8/2018 13:44,42000000,,7000000,...,8,Colombo 8,Borella,6.914720,79.877800,6.912180,79.882881,Colombo 8,0.0,Karshue Avenue
1,251172,17540,NaN,Battaramulla,56.7P Land Directly Overlooking Parliment Grou...,?Our Reference No. SL 8551<br />\n<br />\n(Ple...,6/25/2018 10:26,141750000,,2500000,...,1710,Battaramulla,NaN,6.898378,79.922256,6.897994,79.922287,Battaramulla,54.0,NaN
2,253432,71194,Cotta Road,Colombo 8,Land for sale in Borella,The perfect plot of land for your business or ...,7/12/2018 14:46,65000000,,7926829,...,8,Colombo 8,Borella,6.914720,79.877800,6.912180,79.882881,Colombo 8,0.0,Cotta Road
3,254226,71194,Clifford place,Colombo 4,Large plot of land at Colombo 04,The perfect location to build your dream home ...,7/19/2018 12:16,297000000,,10000000,...,4,Colombo 4,Bambalapitiya,6.896134,79.857147,6.896134,79.857147,Colombo 4,0.0,Clifford place
4,254241,71194,Clifford place,Colombo 4,Plot of land for sale at Colombo 04,The perfect plot of land for your business or ...,7/19/2018 13:44,99000000,,10000000,...,4,Colombo 4,Bambalapitiya,6.896134,79.857147,6.896134,79.857147,Colombo 4,0.0,Clifford place
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20522,5526489,193404,NaN,Piliyandala,Land For Sale In Piliyandala,අයෝජනයටත් පදිංචියටත් කදිම බිම් කොටස් කිහිපයක්....,8/7/2023 9:13,1000000,Per Perch,1000000,...,36,Piliyandala,NaN,6.801757,79.922729,6.801757,79.922729,Piliyandala,7.0,NaN
20523,5527062,193404,NaN,Kesbewa,Best Plots for sale-Kesbewa,කැස්බෑව නගරයට අතේ දුරින් <br />\n<br />\nනව ඉඩ...,8/8/2023 10:07,1000000,Per Perch,1000000,...,73,Kesbewa,Kesbawa / Kasbawa,6.778670,79.947250,6.778670,79.947250,Kesbewa,60.0,NaN
20524,5528131,193404,NaN,Piliyandala,Luxury Land for Sale -Piliyandala,"පිලියන්දල,කැස්බෑව නගරයන්ට මැදිව <br />\n<br />...",8/9/2023 3:40,1000000,Per Perch,1000000,...,36,Piliyandala,NaN,6.801757,79.922729,6.801757,79.922729,Piliyandala,7.0,NaN
20525,5410000,53682,Hospital Road,Dehiwala,Land for Sale in Dehiwela Kalubowila,"Land for Sale in Dehiwela Kalubowila, 19.5p, P...",1/3/2023 8:45,4000000,Per Perch,4000000,...,24,Dehiwala,Dehiwela,6.830119,79.880081,6.830119,79.880081,Dehiwala,4.0,Hospital road


In [6]:
output_csv_path = '../Dataset/New_Data/unique_streets_with_other_data.csv'
df2.to_csv(output_csv_path, index=False)

In [7]:
# Missing value count by column
missing_counts_raw = df2.isnull().sum()

table_data = []
for column, count in missing_counts_raw.items():
    table_data.append([column, count])

print(tabulate(table_data, headers=["Column Name", "missing_counts_raw"], tablefmt="psql"))

+---------------------+----------------------+
| Column Name         |   missing_counts_raw |
|---------------------+----------------------|
| ad_id               |                    0 |
| UID                 |                    0 |
| street              |                 6474 |
| city                |                    0 |
| heading             |                    0 |
| desc                |                    0 |
| posted_date         |                    0 |
| price               |                    0 |
| price_type          |                  414 |
| price_land_pp       |                    0 |
| price_land_pa       |                    0 |
| price_land_total    |                    0 |
| agent_ref           |                10870 |
| contact_type        |                   36 |
| contact_name        |                  338 |
| email               |                    3 |
| price_validate      |                16483 |
| dist_station        |                 8871 |
| dist_bussto

In [8]:
# Get the number of unique values per column
unique_counts = df2.nunique()

table_data = []
for column, count in unique_counts.items():
    table_data.append([column, count])

print(tabulate(table_data, headers=["Column Name", "Unique count"], tablefmt="psql"))

+---------------------+----------------+
| Column Name         |   Unique count |
|---------------------+----------------|
| ad_id               |          20527 |
| UID                 |           2786 |
| street              |           5387 |
| city                |            243 |
| heading             |          15167 |
| desc                |          17565 |
| posted_date         |          20251 |
| price               |           1237 |
| price_type          |              7 |
| price_land_pp       |           1345 |
| price_land_pa       |            185 |
| price_land_total    |           3867 |
| agent_ref           |           6243 |
| contact_type        |              5 |
| contact_name        |           3065 |
| email               |           2880 |
| price_validate      |              7 |
| dist_station        |            111 |
| dist_busstop        |            138 |
| size_land           |            932 |
| land_units          |              2 |
| land_perch    